<a href="https://colab.research.google.com/github/in7erval/HelpWeb/blob/master/face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для начала установим и подключим необходимую библиотеку

In [ ]:
!pip3 install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=6acd181e9634b8e3a301e745b40ca93fc2ff7d306e0c64b9e907a579f283ea98
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import face_recognition as det           # основная библиотека
from PIL import Image, ImageDraw   # библиотека для работы с изображениями
import numpy as np                          # математическая библиотека для удобной работы с массивами (как в Matlab)


Вытащим изображения в текущий репозиторий

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt

Mounted at /content/gdrive


In [ ]:
!cp gdrive/MyDrive/images/*.jpg .

Зададим название изображения для распознавания

In [ ]:
image_name = "manyfaces.jpg"

Выполним поиск лиц на изображении с помощью гистограммы направленных градиентов (hog)

In [ ]:
image_hog = det.load_image_file(image_name)
face_locations_hog = det.face_locations(image_hog)
face_locations_hog

Покажем лица, выделив их на изображении

In [ ]:
img = Image.open(image_name)
for face_location in face_locations_hog:
  top, right, bottom, left = face_location
  ImageDraw.Draw(img).rectangle((left, top, right, bottom), outline='blue', width=2)
img

Выполним поиск лиц на изображении с помощью сверточной сети

In [ ]:
image_cnn = det.load_image_file(image_name)
face_locations_cnn = det.face_locations(image_cnn, model='cnn')
face_locations_cnn

И так же выделим их на изображении

In [ ]:
img = Image.open(image_name)
for face_location in face_locations_cnn:
  top, right, bottom, left = face_location
  ImageDraw.Draw(img).rectangle((left, top, right, bottom), outline='blue', width=2)
img

Видим, что свёрточная нейронная сеть работает более точно, но медленней. 

В методе можно регулировать величину увеличения разрешения (по умолчанию = 1). Рассмотрим другой пример с более мелкими лицами. Для начала оставим всё как было.

In [ ]:
image_name = "9may.jpg"
img = Image.open(image_name)
img

In [ ]:
face_locations_hog = det.face_locations(np.array(img), model='hog')
for face_location in face_locations_hog:
  top, right, bottom, left = face_location
  ImageDraw.Draw(img).rectangle((left, top, right, bottom), outline='blue', width=2)
face_locations_hog

Видно, что программа не нашла ни одного лица на фото. Увеличим разрешение в 3 раза.

In [ ]:
img = Image.open(image_name)
face_locations_hog = det.face_locations(np.array(img), number_of_times_to_upsample=2, model='hog')
len(face_locations_hog)

Нашлось целых вот столько лиц ^ . ( Если запускать несколько раз, будет получаться разное количество, т.к. веса изначально задаются рандомно). Прогоним поиск лиц 10 раз, чтобы найти максимальное количество лиц на фото. (Занимает примерно 5 минут, так что надо ждать..)

In [ ]:
max_faces_found = 0
max_faces = list()
img = Image.open(image_name)
for i in range(10):
  face_locations_hog = det.face_locations(np.array(img), number_of_times_to_upsample=3, model='hog')
  cur_faces_found = len(face_locations_hog)
  if cur_faces_found > max_faces_found:
    max_faces_found = cur_faces_found
    max_faces = face_locations_hog
print('Нашлось лиц: %d' % max_faces_found)

Для наглядности покажем найденные лица на фото

In [ ]:
img = Image.open(image_name)
for face_location in max_faces:
  top, right, bottom, left = face_location
  ImageDraw.Draw(img).rectangle((left, top, right, bottom), outline='yellow', width=2)
img